# Semantic Style GAN のモーフィング生成

## 1. 事前学習済みモデルをダウンロードする
学習済みモデルをpklファイルをダウンロードして`pretrained/` に配置する．`pretrained/`がない場合は自分で作成すること．<br>
**DL : https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/CelebAMask-HQ-512x512.pt**

## 2. 画像のサイズを事前学習済みモデルに合わせる
CelebAMask-HQでは512x512サイズであるので，入力画像もそのサイズにリサイズしてBGR形式を作成する．<br>
`images/`がない場合は作成し入力画像を配置して以下のプログラムを動かす．元画像が上書きされるので注意が必要．

In [ ]:
!python resize512.py --indir images

## 3. 顔画像を中心に配置
少しでも顔が中心でないく正しく推論できないため，画像を中心に配置する．

In [ ]:
!python align_images.py --indir images

## 4. 出力フォルダを作成する

invertするときの出力先は`results/inversion`，モーフィングの出力先は`results/interpolation`とする．

In [ ]:
mkdir -p results/inversion results/interpolation

## 5. 画像をモデルにinvertする（学習済みモデルに則して潜在変数を探索してファインチューニングをする）
`images/`にある顔画像でをinvertして潜在変数(npy)を作成して，モデル(pt)をファインチューニングする．ファインチューニングはptiアルゴリズムを用いる．<br>
潜在変数は`results/inversion/latent/`，ファインチューニング済みモデルは`results/inversion/weights/`に出力される．

In [ ]:
!PYTHONPATH=.:$PYTHONPATH python visualize/invert_pti.py --ckpt pretrained/CelebAMask-HQ-512x512.pt --imgdir images --outdir results/inversion --size 512 --step 3000  --finetune_step 5000

### (5.2) 潜在変数を確認する
潜在変数が本当に指定した顔に近づいているか確認する．<br>
顔画像だけなら`results/inversion/recon/`，`results/inversion/recon_finetune/`に出力されるが，パーツごとの画像やスペクトルなどを調べるには以下を使う．

In [ ]:
!python components_display.py

## 6. モーフィングを作成する
`/pretrained`にモデルを移してモーフィングを作成する．<br>
逆順のモーフィングは作成されないのが逆再生で対応する．顔パーツ区間に分けながら潜在変数で線形モーフィングする．<br>
線形モーフィングは $ I^M=\alpha I^S+(1.0-\alpha) I^T \quad(0 \leq \alpha \leq 1.0)$ ではあるが，$ I^S $，$ I^T $は単に画素値ではなく潜在変数であるこに留意する．

In [ ]:
cp -f results/inversion/weights/pti.pt pretrained/

In [1]:
!python morphing_line.py

pretrained/pti.pt
Loading model ...
n_latent: 28, n_latent_expand: 130
Generating original image ...
Generating morphing AN -> DI
100%|#########################################| 100/100 [00:33<00:00,  2.98it/s]
Loading model ...
n_latent: 28, n_latent_expand: 130
Generating original image ...
Generating morphing AN -> FE
100%|#########################################| 100/100 [00:33<00:00,  2.97it/s]
Loading model ...
n_latent: 28, n_latent_expand: 130
Generating original image ...
Generating morphing AN -> HA
100%|#########################################| 100/100 [00:29<00:00,  3.35it/s]
Loading model ...
n_latent: 28, n_latent_expand: 130
Generating original image ...
Generating morphing AN -> LFAU09
100%|#########################################| 100/100 [00:31<00:00,  3.18it/s]
Loading model ...
n_latent: 28, n_latent_expand: 130
Generating original image ...
Generating morphing AN -> LFAU10
100%|#########################################| 100/100 [00:35<00:00,  2.81it/s]
Loading m

## memo
- 3において2をおこなわないと色空間の影響で上手に配置できないときがある．